In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding, Dropout
from tensorflow.keras.datasets import imdb
import numpy as np
from keras.datasets import reuters
from keras.utils.np_utils import to_categorical

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
# (x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
#                                                          num_words=None,
#                                                          skip_top=0,
#                                                          maxlen=None,
#                                                          test_split=0.2,
#                                                          seed=113,
#                                                          start_char=1,
#                                                          oov_char=2,
#                                                          index_from=3)

(X_train_seq, Y_train), (X_test_seq, Y_test) = reuters.load_data()

In [3]:
vocabulary_size = -1

for seq in X_train_seq:
    max_index = max(seq)
    if max_index > vocabulary_size:
        vocabulary_size = max_index

for seq in X_test_seq:
    max_index = max(seq)
    if max_index > vocabulary_size:
        vocabulary_size = max_index

vocabulary_size += 1

X_train = sequence.pad_sequences(X_train_seq, maxlen=100)
X_test = sequence.pad_sequences(X_test_seq, maxlen=100)

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [6]:
print(np.shape(X_train_seq), np.shape(Y_train), np.shape(X_test_seq), np.shape(Y_test))
print(np.shape(X_train_seq[0]), np.shape(X_test_seq[0]))
print(np.shape(X_train), np.shape(X_test))

(8982,) (8982, 46) (2246,) (2246, 46)
(87,) (145,)
(8982, 100) (2246, 100)


In [14]:
model = Sequential()
model.add(Embedding(output_dim=300, input_dim=vocabulary_size, input_length=100))

# model.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(46, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()
model.fit(X_train, Y_train, epochs=10, batch_size=128)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          9294600   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 100, 128)          115328    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 50, 128)           0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 50, 64)            24640     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 25, 64)            0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 25, 64)           

In [15]:
model.evaluate(X_train,Y_train)

281/281 [==============================] - 1s 5ms/step - loss: 0.1594 - accuracy: 0.9582


[0.1593668907880783, 0.9582498073577881]

In [16]:
model.evaluate(X_test,Y_test)

71/71 [==============================] - 0s 4ms/step - loss: 2.1951 - accuracy: 0.6336


[2.1950700283050537, 0.6335707902908325]